# Resolution in Python

**Outline**:
- Lookup mechasim
- Inheritance
- Descriptor
- Monkey Patching
- Misc & closing words

## Lookup mechanism

When accessing the attribute of an object, Python will proceed as follow:
1. invoke the `__getattribute__` method, which is responsible for
2. looking into the instance `__dict__` for the attribute
3. looking into the class `__dict__` if the former fails
4. delegate to the `__getattr__` method for custom handling if the former fails
5. if `__getattr__` is not implemented, an `AttributeError` is raised.

This lookup will be refined as we introduced new concepts.

> :warning: :skull: Note that if the class is defined with `slot`, the instance `__dict__` will be missing (unless it is explictly re-created as a slot).

Let's invsetigate the lookup step by step.

In [ ]:
# Attribute lookup
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)



r = Rectangle(10, 20)
print(r.width)
print(r.__dict__)

print("---")
r.width = 15
print(r.__dict__)

> :warning: Note that Python is not using the `__getitem__` method of the dictionary (at least in CPython) but does a more direct lookup in the underlying C implementation.

In the case of a class attribute, or method, we end up in the **class** `__dict__`

In [ ]:
# Method and class attribute lookup
class Rectangle:

    _AUTHOR_ = "JMB"

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)

    def area(self):
        return self.width * self.height


r = Rectangle(10, 20)
print(r.__dict__)  # Only attributes

print("---")
r._AUTHOR_ # goes through the __getattribute__ method
r.area()   # goes through the __getattribute__ method

print("---")
print(Rectangle.__dict__)  # Contains _AUTHOR_ and area

Interestingly, when performing a lookup for a method, Python finds it in the **class** `__dict__` and then binds it to the instance. This is different from the following

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        def area():
            return self.width * self.height
        self.area = area

r = Rectangle(10, 20)
print(r.__dict__)  # Contains area as well


## Inheritance

### Simple inheritance

What happens in case of inheritance?

In [ ]:
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height
    

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)
    

print(Square.__dict__)  # area not in Square.__dict__
print(Square.area)  # Accessible even though it is not in Square.__dict__

:question: Consider the following piece of code. Explain in which `__dict__` will the following be found (if any)
- `__o.a_value`
- `__o.b_value`
- `__o.a_method_1`
- `__o.a_method_2`
- `__o.b_method`
- `__o.CONSTANT`
- `__o.CONSTANT_2`
- `__o.c_method`

In [ ]:
class A:
    CONSTANT = 3.14
    CONSTANT_2 = 42

    def __init__(self, a_value):
        self.a_value = a_value

    def a_method_1(self):
        pass

    def a_method_2(self):
        pass

class B(A):
    CONSTANT = 6371

    def __init__(self, a_value, b_value):
        super().__init__(a_value)
        self.b_value = b_value

    def b_method(self):
        pass

    def a_method_1(self):
        pass

__o = B(10, 750)
__o.c_method = lambda: -1

### Multiple inheritance and MRO


Python allows for Multiple Inheritance. An issue that arises from such mechanism is called the Diamond problem. Consider the following inheritance diagram:
```
      Person        
     /      \       
Teacher  Researcher 
     \      /       
    Professor       
```
What happens if a method of a `Professor` instance is called when
- the method is defined only in `Professor`;
- the method is defined only in `Person`;
- the method is re-defined both in `Teacher` and `Researcher`?

The most common such method is the `__init__` dunder.

In [ ]:
class Person:
    def __init__(self):
        print("Person init")

class Teacher(Person):
    def __init__(self):
        print("Teacher init")
        super().__init__()

class Researcher(Person):
    def __init__(self):
        print("Researcher init")
        super().__init__()


In [ ]:
class Professor1(Researcher, Teacher):
    def __init__(self):
        print("Professor init (R, T)")
        super().__init__()

_ = Professor1()

In [ ]:
class Professor2(Teacher, Researcher):
    def __init__(self):
        print("Professor init (T, R)")
        super().__init__()

_ = Professor2()

Python uses the C3 linearization algorithm to determine the orders of the calls. In practice, a deterministic order is established and stored in the `__mro__` (Multi-inheritance Resolution Order) field of the **class**, also accessible via the `mro` class method:

In [ ]:
print("Professor 1:", Professor1.__mro__, Professor1.mro())
print("Professor 2:", Professor2.__mro__)

the `super` call follows the MRO. See the "Advanced Inheritance" chapter for more on multiple inheritance.

> Note how the last one if always the `object` base class.

> :warning: The C3 algorithm is neither a breadth-first, nor depth-first exploration of the hierarchy DAG. Rather it is a merging algorithm which aims to enforce three poperties (local precedence, monotonicity and consistency).

Even in the case of simple inheritance, Python relies on the MRO for the lookup, although it is trivial then:

In [ ]:
Square.mro()

:wrench: Exercise: determine the output of `G().greet()`. You can use the MRO.

In [ ]:
class A:
    def greet(self):
        return "Hello from A"
    
class B(A):
    def greet(self):
        return "Hello from B"

class C(A):
    def greet(self):
        return "Hello from C (property)"

class D(B, C):
    def greet(self):
        return super().greet() + " @ D"

class E:
    def greet(self):
        return "Hello from E"
    
class F(E, C):
    pass

class G(D, F):
    pass

# G.mro()

## Fallback `__getattr__`

The last step of the resolution in case of miss is to invoke the `__getattr__` method, which by default raises a `AttributeError` exception. 

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def __getattr__(self, name):
        print(f"__getattr__ called for: {name}")
        if name == "perimeter":
            return lambda: 2*self.width + 2*self.height  # Note that self is not passed
        raise AttributeError(f"{name} not found (__getattr__)")
    
r = Rectangle(10, 20)
print("Width:", r.width)
print("---")
print("Area:", r.area())
print("---")
print("Perimeter:", r.perimeter())
print("---")
print(r.radius)

Python exposes the `getattr` method to call the `__getattr__` dunder, with the nice addition of possibly returning a default value:

In [ ]:
getattr??

:wrench: Write a `LogCallDecorator` class, which prints every call made to the decorated object but otherwise behaves exactly as the decorated object.

In [ ]:
from typing import Any


class LogCallDecorator:
    def __init__(self, __o: Any):
        self.__o = __o
    
    # TODO 

There are several cases where overriding the `__getattr__` method is handy, mostly in advance cases. This include mocking, lazyness, and design patterns (eg. decorator, builder, vistitor).

> :pushpin: In Hermes, `__getattr__` is used the `HermesEstimatorBuilder`, and in the visitors for the filters

### `__getattribute__` vs `__getattr__`, when to use which?

Although close, `__getattribute__` and `__getattr__`  serve different purposes. The former is a *interceptor*, occuring at the start of the lookup. The latter is an *fallback*, occuring at the end of the lookup.

`__getattribute__` drives the resolution process. As such, it can be used as hook to provide some pre-/post-processing. The super call is almost-always invoked, otherwise the lookup of every attribute is impacted. It is typically used for
- wrappers/hooks for log, audit, sandbox, etc;
- access control (over all attributes)

`__getattr__` is triggered only as last resort. As such, it does not impact the whole resolution (no need to invoke super, but should raise an `AttributeError` as default) and it will not impact the resolution of other attributes (safe). It is typically used for
- dynamic attribute/lazy load (on-demand computation);
- delegation (cf. Decorator).

:thumbsup: rule: use `__getattribute__` only when `__getattr__` is not sufficient.

> dynamic attribute and lazy loading are often implemented via decoration and descriptor. 


> :bulb: remember that in `__getattribute__` there is a `i` like in interceptor. `__getattr__` is shorter, like some letters fell out-of-it.

## Descriptor

### The notion of descriptor

It is time to revisit the lookup thanks to the notion of descriptors:
1. invoke the `__getattribute__` method, which is responsible for
2. looking into the instance `__dict__` for the attribute
3. looking into the class `__dict__` if the former fails
4. **Apply descriptor if something is found** else
5. delegate to the `__getattr__` method for custom handling  if the former fails
6. if `__getattr__` is not implemented, an `AttributeError` is raised.

Descriptors let objects customize attribute lookup, storage, and deletion (https://docs.python.org/3/howto/descriptor.html). This is a powerful mechanism. Technically, a descriptor is any object which defines `__get__`, `__set__`, or `__delete__` method.

> :warning: the descriptor is only applied as a class attribute (not instance, nor in `__getattr__`). 

In [ ]:
import datetime as dt
from time import sleep

class Now:
    def __get__(self, instance, owner):
        print("__get__ called")
        return dt.datetime.now()
    
class MyClass:
    now = Now()


obj = MyClass()
print(obj.now) 
sleep(10)
print(obj.now)  

It is often useful for a descriptor to know the name it is associated with. Python allows to do that via the `__set_name__` dunder. 

In [ ]:
class Tracker:
    def __init__(self, value):
        self.value = value
        self.name = None

    def __set_name__(self, owner, name):
        # print(f"Setting name: {name}")
        self.name = name

    def __get__(self, instance, owner):
        print(f"accessing {self.name if self.name else 'unknown'}")
        return self.value
    
class MyClass:
    x = Tracker(10)
    y = Tracker(20)
    # z = x  # overrides name


obj = MyClass()
print(obj.x)
print(obj.y)
print(obj.x)

### Properties

In the examples above, notice how we are just accessing, not calling. This behavior should be reminiscent of `property`. Indeed `property` relies on the descriptor mechanism (and function decorator):

In [ ]:
def property_(f):
    class _Property:
        def __init__(self, func):
            self.func = func
            self.__doc__ = getattr(func, "__doc__", None)

        def __get__(self, instance, owner):
            if instance is None:
                return self  # like built-in property when accessed on the class
            if self.func is None:
                raise AttributeError(f"unreadable attribute")
            return self.func(instance)

    return _Property(f)


class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property_
    def pi(self):
        return 3.14

    def area(self):
        return self.pi * self.radius ** 2



c = Circle(10)
print(c.pi)  # simple access, but the property_ does the call

> Similarly, classmethod and staticmethod (and other mecanism) are based on descriptor (see https://docs.python.org/3/howto/descriptor.html#pure-python-equivalents for more)

### Stateful descriptor, setting and sharing

:wrench: Create a `Rate` descriptor which validates that the associated value is a float between 0 and 1. You can set the value of descriptor via assignment through the `__set__(self, instance, value)` dunder.

In [ ]:
class Rate:
    pass

class MyClass:
    rate = Rate(0.1)

obj = MyClass()
print(obj.rate)
print("---")
obj.rate = 0.5
print(obj.rate)

One of the issue with the code above is that `rate` must be a class variable for the descriptor to opperate:

In [ ]:
class MyClass:
    def __init__(self, rate):
        self.rate = Rate(rate)

MyClass(.5).rate  # Does not return the value, but the descriptor object

As such, the descriptor is shared across all instances:

In [ ]:
class MyClass:
    rate = Rate(0.1)

obj1 = MyClass()
obj2 = MyClass()
obj2.rate = 0.8
obj1.rate

One common way to isolate the descriptor state from several instances, is to store the state in the instance directly. Let's illustrate that with a `NominalValidator`

In [ ]:
class NominalValidator:
    def __init__(self, *allowed_values):
        self.allowed_values = allowed_values

    def __set_name__(self, owner, name):
        self.key = "_" +  name  # avoid name clash and store it as protected/private

    def __get__(self, instance, owner):
        return getattr(instance, self.key)

    def __set__(self, instance, value):
        if value not in self.allowed_values:
            raise ValueError(f"{value} not allowed")
        return setattr(instance, self.key, value)
    
class Person:
    favorite_color = NominalValidator("red", "green", "blue")

    def __init__(self, name, favorite_color):
        self.name = name
        self.favorite_color = favorite_color

alice = Person("Alice", "red")
bob = Person("Bob", "blue")
# charlie = Person("Charlie", "purple")  # raises ValueError
print(alice.favorite_color)
print(bob.favorite_color)
print("---")
print(alice.__dict__)  # Value has been added to __dict__

Alternatively, the state can be stored on the descriptor side by associating the instances in a dictionary:

In [ ]:
from weakref import WeakKeyDictionary

class NominalValidator:
    def __init__(self, *allowed_values):
        self.allowed_values = allowed_values
        self._data = WeakKeyDictionary()

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._data.get(instance)

    def __set__(self, instance, value):
        if value not in self.allowed_values:
            raise ValueError(f"{value} not allowed")
        self._data[instance] = value
    
class Person:
    favorite_color = NominalValidator("red", "green", "blue")

    def __init__(self, name, favorite_color):
        self.name = name
        self.favorite_color = favorite_color

alice = Person("Alice", "red")
bob = Person("Bob", "blue")
print(alice.favorite_color)
print(bob.favorite_color)
print("---")
print(alice.__dict__)  # Value not in __dict__

:warning: Some important remarks
- Storing the state in the instance of course risks running into name collisions.
- Using `getattr` and `setattr` when using the instance to store the state might result in weird bug in case of re-implementation of `__getattr__` and `__setattr__`.
- An alternative to `getattr` and `setattr` is to use the `__dict__` directly (which does not exist in case of slots).
- The "weakref" approach is cleaner in terms of encapsulation but it breaks cloning and serialization.

Overall, the most Pythonic idioms is to use store the state in the instance `__dict__` (directly, or indirectly via `getattr`/`setattr`).

:question: descriptors require a lot of logic to manage a state, which could be handled in a more standard way. When or why are they useful?

:wrench: Write a `Undoable` descriptor. The field remembers previous values. A special value can be assigned to the field to go back to the previous value.

In [ ]:
class Undoable:
    pass


class Person:
    job = Undoable()

    def __init__(self, name):
        self.name = name


alice = Person("Alice")
alice.job = "Researcher"
print(alice.job)
alice.job = "Professor"
print(alice.job)
alice.job = Undoable.Previous
print(alice.job)

## Monkey Patching

> :monkey: with great power comes great complexity

:bulb: The term Monkey Patching comes from the idea of changing the monkey while the circus is running.

### Dynamic assignment

Python allows for dynamic assignments; adding things to an existing instance at run time:


In [ ]:
class Person:
    def __init__(self, name):
        self.name = name

alice = Person("Alice")
alice.age = 30
setattr(alice, "favorite_color", "blue")
print("Age:", alice.age)
print("Color:", alice.favorite_color)
print(alice.__dict__)  # stored in __dict__

Some remarks:
- the dot+assigment operation and the `setattr` call both invoke the `__setattr__` dunder of the instance;
- `__setattr__` can be overriden (rarely done);
- as part of the implementaton of `__setattr__`, the `__set__` method of a descriptor is invoked; there is no `__setattribute__` method;
- objects with slots do not support dynamic assigment.

Dynamic assignment is not limited to variables:

In [ ]:
# Changing a method
class Person:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print(f"Hello, my name is {self.name}")

alice = Person("Alice")
alice.say_hello()  # Method call, self is passed automatically
print(Person.__dict__["say_hello"])  # The lambda function
print("---")

Person.say_hello = lambda self: print(f"Hi, I'm {self.name}")
alice.say_hello()
print(Person.__dict__["say_hello"])  # The lambda function

In [ ]:
# Biding a method to an instance
import types

bob = Person("Bob")
bob.say_hello() 
print("---")
bob.say_hello = types.MethodType(lambda self: print(f"Yo, {self.name} here!"), bob) # Notice bob is passed for binding
bob.say_hello() 
print(bob.__dict__["say_hello"])  # The bound method is stored in __dict__
print("---")
alice.say_hello()  # Alice still uses the class method

Although the two example above result in changing dynamically the behavior of `say_hello`, the results are very different!

In the first example, the class `__dict__` is modified. As a result, all the instances are affected. This is even true of the created instances, due to how the look up works. This is a major side effect.

In the second example, only the instance `__dict__` is affected (note the use of `MethodType` and the biding of the actual instance). The effect is much more limited.

> :warning: it is possible to patch modules as well, but the import statement caches the module in `sys.modules`. It is possible to use `importlib.relead` to re-run the module's code, but pre-existing reference to the old module will remain. Patching eg. classes works because of the dynamic lookup.

> :skull: it is even possible to dynamically (re-)assign a class to an instance; although maintaining consistency with inheritance (notably) is tricky.

### Gardrails & patterns

Dynamic assigment is Pythonic and powerful, but it can produce major side effects and render the code hard to understand and maintain. It should be limited in its use. Eg.:
- patching code that is outside of the code base;
- used in controlled environment such as in tests (cf. `pytest.mock`), or short-lived code.

A common pattern is to do chain aliasing (name coming from Ruby). This consist in remembering the previous value so as to be able to restore it. This combines well with a context manager to limit the scope of the change and automatically restore the previous version.

:wrench: Create a context manager to perform chain alisasing instance attribute.

In [ ]:
from contextlib import contextmanager
from typing import Any

@contextmanager
def alias(obj: Any, attr_name: str, new_value: Any):
    # TODO remember the value, set the new value and restore the old value
    raise NotImplementedError("You need to implement this")

class Person:
    def __init__(self, name):
        self.name = name

alice = Person("Alice")
with alias(alice, "name", "bob"):
    print(alice.name)  # bob



Another interesting use case is in-place dynamic decoration. Since the decoration is not supposed to change the behavior of the decorated object, side effects should be limited. This allows to use decorators where you cannot change the code definition directly. It also limits the scope.

In [ ]:
import functools

def logit(func):
    # Standard decorator
    @functools.wraps(func)  
    def wrapper(*args, **kwargs):
        print(f"> Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"> {func.__name__} done.")
        return result
    return wrapper

class Person:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print(f"Hello, my name is {self.name}")


alice = Person("Alice")
alice.say_hello = logit(alice.say_hello)  # Dynamic decoration
alice.say_hello()  # Now it is logged
print("---")
print(alice.__dict__)

In [ ]:
# Tips, can be used for debugging
class Person:
    def __init__(self, name):
        self.name = name

    def say_hello(self):
        print(f"Hello, my name is {self.name}")


alice = Person("Alice")
Person.__getattribute__ = logit(Person.__getattribute__)
alice.say_hello()

## Misc

### `hasttr`

Besdies `getattr` and `setattr`, there exists `hasattr` which determines whether an instance has a given attribute. Note that it is implemented as 
```Python
def hasattr(obj, name)
    try:
        getattr(obj, name)
    except AttributeError:
        return False
    return True
```

which means the `__getattr__` dunder is called. In case of re-implementation, some code might be run unintended. In particular, if the subsequent code accesses the attribute, the code goes twice through the lookup. It is also possible that an exception other than a `AttributeError` is raised, exiting the code with a non-boolean resolution. 

It is therefore encouraged to use `getattr` directly.


### `dir` / `vars`

`dir`, `vars` and `__dict__` are often seen in the same context, but differ in the following way:

| Tool        | What it shows                                                             | Output type                          |
| ----------- | ------------------------------------------------------------------------- | ------------------------------------ |
| `__dict__`  | Actual stored attributes on the object (instance/class).                  | `dict` (or `mappingproxy` for class) |
| `vars(obj)` | Shortcut for `obj.__dict__` (if it exists); otherwise `TypeError`.        | dict / mappingproxy                  |
| `dir(obj)`  | All attribute names available via lookup (instance dict + class + bases). | `list[str]`                          |


### Slots

As we have seen, common classes in Python rely on a dictionary to store instance's state. This allows for flexibility and dynamic assignments. Python allows for another type of classes however, where instead of a dictionary a fixed portion of memory is reserved. The latter allows for fewer memory waste and much faster memory access. As we have seen in this training, those objects break some assumptions, and the general tip is to only use them when necessary.

> See https://docs.python.org/3.10/reference/datamodel.html?highlight=slots#object.__slots__ for more.


## :wrench: Open question

I would like to log all the access to the public methods (and only those) of a class which is in library outside of the codebase. How would you do that?

Bonus exercise: implement your idea.



## Closing words

This training was concerned about the lookup mechanism in Python, how it interplays with inheritance, as well as what it allows for in term of dynamic assignment. It allowed us to discuss powerful tools (like descriptors) and techniques (like monkey patching), as well as when to--and not to--use them.